# Testing and validation approaches for scientific software

![The perils of floating point](img/floating-point.png)

### Juan Luis Cano - 2018-09-25 OSCW '18 @ ESAC, Madrid

# Who am I?

<img src="img/juanlu_esa.jpg" alt="Me!" width="400" style="float:right"></img>

* **Aerospace Engineer** with a passion for orbits 🛰
* Chair of the **Python España** non profit and co-organizer of **PyCon Spain** 🐍
* **Software Developer** at **Satellogic** 🌍
* Free Software advocate and Python enthusiast 🕮
* Hard Rock lover 🎸

Follow me! https://github.com/Juanlu001/

<div style="clear:both"></div>

# Introduction to poliastro

<img src="img/logo_text.svg" alt="poliastro" width="500" style="float:right"></img>

> poliastro is Python library for Astrodynamics and Orbital Mechanics, focused on interactive and friendly use and with an eye on performance.

* **Pure Python**, accelerated with **numba**
* **MIT license** (permissive)
* Physical units, astronomical scales and more, thanks to Astropy
* Conversion between several orbit representations
* Analytical and numerical propagation
* Cool documentation 🚀 http://docs.poliastro.space/
* Latest version 0.11 released last Friday!

<div style="clear:both"></div>

## Brief history

* _2013_: First version: Octave + FORTRAN + Python
* _2014_: Refactor of the API, much friendlier
* _2015_: Replace FORTRAN algorithms by Python + numba 🚀
* _2016_: Izzo algorithm (Lambert's problem), 6th ICATT @ ESA
* _2017_: Summer of Code in Space (SOCIS), OpenAstronomy & Astropy membership, 1st OSCW @ ESOC
* _2018_: Google Summer of Code (GSOC), #PyAstro18 @ Simons Fndn, expansion into the industry

# Challenges

## Validation

> Unit testing a function with clear expectations is trivial. What are my expectations on numerical algorithms?

The wrooooooooooooooong way:

In [15]:
def sinc(x):
    return np.sin(x) / x

In [16]:
import pytest

In [17]:
@pytest.mark.parametrize("x", [0, 1, 10])
def test_sinc(x):
    assert sinc(x) == np.sin(x) / x

In [18]:
0.1 + 0.2 == 0.3

False

In [19]:
0.2 + 0.3 == 0.5

True

A better way:

* Compare against some authoritative source: **external data or software**
* Do floating point comparisons right and **use tolerances**

In [2]:
def test_convert_from_rv_to_coe():
    # Data from Vallado, example 2.6
    attractor = Earth
    p = 11067.790 * u.km
    ecc = 0.83285 * u.one
    inc = 87.87 * u.deg
    raan = 227.89 * u.deg
    argp = 53.38 * u.deg
    nu = 92.335 * u.deg
    expected_r = [6525.344, 6861.535, 6449.125] * u.km
    expected_v = [4.902276, 5.533124, -1.975709] * u.km / u.s

    r, v = ClassicalState(attractor, p, ecc, inc, raan, argp, nu).rv()

    assert_quantity_allclose(r, expected_r, rtol=1e-5)
    assert_quantity_allclose(v, expected_v, rtol=1e-5)

What's the perfect way?

* How much precision do you ask for? Should you carry a mathematical analysis?
* What if your results don't match? Sometimes, book or paper authors respond to your comments... And sometimes don't
* The changes in precision are a result of bad data, or worse algorithms?
* How do you even track _improvements_?

### External data (short summary)

* Nobody cares

* Those who care, don't share it

* Those who share, do it with 1 decimal place (true story)

* Those who share with 16 decimal places, don't describe how it was obtained (i.e. release the source)

* Those who release the source, make it impossible to compile

### External software

* Sometimes commercial
* Is it validated itself? (See above)
* It is often difficult to reproduce the exact setting and algorithms, most of the times because your commercial software is much more complex

![Shrug](img/shrugging-guy.jpg)

...If you're really interested, go read my Final Masters Project: https://github.com/juanlu001/pfc-uc3m

## Performant and "for humans"

* _Yes, Python is slow_ (compared to compiled languages)
* Some algorithms cannot easily be vectorized (i.e. replaced by NumPy)
* And even if you can, vectorized code can be impossible to read
* I don't like Cython (messy, impossible to debug, I don't know C)
* _numba helps only with Fortran-esque code_ (forget about closures or introspection)

![Too smart](img/too_smart.png)

## Is it that bad?

<img src="img/mco.png" alt="Mars Climate Orbiter" width="600" ></img>

So... let's make our code Fortran-esque!

<img src="img/architecture.svg" alt="Architecture" width="500" style="float:right"></img>

High level API:

* Supports mixed units and time scales, figures out the rest
* Easy to use and impossible to get wrong
* **Slow**

Dangerous™ algorithms:

* **Fast** (easy to accelerate with numba or Cython)
* Only cares about numbers, makes assumptions on units (SI, TBD)
* **You can mess it up**

<div style="clear:both"></div>

### Measure everything!

http://poliastro.github.io/poliastro-benchmarks

![Benchmarks](img/benchmarks.png)

# _Per Python ad astra!_ 🚀

* Slides: https://github.com/poliastro/oscw2018-talk
* poliastro chat: https://riot.im/app/#/room/#poliastro:matrix.org
* Twitter: https://twitter.com/poliastro_py